# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf

warnings.filterwarnings(action='ignore')

In [2]:
with open('./models/midfielder_ss.pickle','rb') as f:
    midfielder_ss = pickle.load(f)
    
with open('./models/midfielder_qt.pickle','rb') as f:
    midfielder_qt = pickle.load(f)

with open('./models/midfielder_mm.pickle','rb') as f:
    midfielder_mm = pickle.load(f)

midfielder_model = load_model('./models/midfielder_nn.h5')

    
premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
midfielder_df = premier_df[premier_df['포지션']=='미드필더']

In [5]:
use_col = ['골','도움','크로스 성공률','파울 획득','반칙 수/90', '기회 창출/90', '경기당 유효 슈팅/90', '경기당 슈팅/90', '인터셉트/90',
       '달린 거리/90분', '공중볼시도/90', '태클 성공률', '경기당 패스 시도/90', '경기당 패스 성공/90',
       '키패스/90', '태클 성공/90', '드리블 성공/90', '헤더 성공/90', '크로스 시도/90','평균획득승점/경기']

In [6]:
outlier_features = ['헤더 성공/90','경기당 슈팅/90','드리블 성공/90','경기당 유효 슈팅/90',
                    '태클 성공/90','크로스 성공률','달린 거리/90분','기회 창출/90']

nonoutlier_features = ['경기당 패스 성공/90','반칙 수/90','파울 획득','경기당 패스 시도/90',
                       '인터셉트/90','공중볼시도/90','크로스 시도/90','태클 성공률','키패스/90']
order_features = ['골','도움']

In [7]:
midfielder_df.loc[:,outlier_features] = midfielder_qt.transform(midfielder_df.loc[:,outlier_features])
midfielder_df.loc[:,nonoutlier_features] = midfielder_ss.transform(midfielder_df.loc[:,nonoutlier_features])
midfielder_df.loc[:,order_features] = midfielder_mm.transform(midfielder_df.loc[:,order_features])

In [8]:
midfielder_df = midfielder_df.reset_index(drop=True)

In [9]:
good_features = ['도움', '경기당 유효 슈팅/90', '경기당 패스 시도/90', '반칙 수/90', '태클 성공/90']

In [10]:
midfielder_df['예측PPG']=midfielder_model.predict(tf.constant(midfielder_df.loc[:,good_features].values))
midfielder_df

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
0,Billy Gilmour,2021-2022,미드필더,노리치,잉글랜드 프리미어 리그,0.272727,0.205882,21/7/1 - 22/5/31,8,13,...,-0.682513,1.152383,1.048535,0.514723,-0.088734,-0.682458,-1.922355,-1.055482,1.00,1.768646
1,Pierre Lees-Melou,2021-2022,미드필더,노리치,잉글랜드 프리미어 리그,0.272727,0.029412,0,8,13,...,0.994872,1.062199,1.018320,-1.394567,-1.220640,-0.747859,-2.307475,-1.039026,0.92,1.150835
2,Todd Cantwell,2021-2022,미드필더,노리치,잉글랜드 프리미어 리그,0.090909,0.264706,0,8,13,...,0.478753,-1.509494,-1.625484,0.697806,-0.635883,1.327500,-0.308666,0.511037,0.92,1.293803
3,Mathias Normann,2021-2022,미드필더,노리치,잉글랜드 프리미어 리그,0.045455,0.058824,21/7/5 - 22/6/30,7,13,...,-1.069602,1.961129,2.177061,-0.714546,-0.286610,-1.985058,-1.129740,-1.173958,0.92,1.305781
4,Max Aarons,2021-2022,미드필더,노리치,잉글랜드 프리미어 리그,0.045455,0.000000,0,8,13,...,0.349724,-0.793842,-0.652564,0.436259,-0.472789,1.275817,-0.442929,0.409016,0.92,1.234546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,Emerson Royal,2021-2022,미드필더,토트넘,잉글랜드 프리미어 리그,0.090909,0.117647,0,19,14,...,0.220694,-1.433856,-1.214561,1.273208,0.295415,1.176532,0.269586,1.715545,1.73,1.307473
141,Tanguy Ndombele,2021-2022,미드필더,토트넘,잉글랜드 프리미어 리그,0.045455,0.147059,0,26,14,...,0.220694,1.092746,1.222270,-0.636082,0.564534,-0.165327,-0.816263,-1.121302,1.92,1.781698
142,Oliver Skipp,2021-2022,미드필더,토트넘,잉글랜드 프리미어 리그,0.045455,0.058824,0,24,13,...,-1.585720,1.492754,1.365791,-1.682268,0.430727,-1.927864,-0.474176,-1.144339,1.89,1.708352
143,Sergio Reguilón,2021-2022,미드필더,토트넘,잉글랜드 프리미어 리그,0.000000,0.117647,0,19,14,...,1.252931,-0.120372,-0.166104,-0.714546,1.110772,0.923344,0.402432,1.587196,1.65,1.683284


In [11]:
midfielder_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
123,Saúl,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,0.409091,0.411765,21/7/5 - 22/6/30,31,6,...,-0.424454,0.960379,0.814369,0.200867,-0.050661,0.062356,0.495165,-1.088392,2.11,2.291662
124,N'Golo Kanté,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,0.272727,0.676471,0,34,10,...,-1.327661,0.972016,0.823434,-0.165298,-0.277827,-0.496519,-1.347629,-0.858021,2.00,2.158232
121,Hakim Ziyech,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,0.227273,0.176471,0,21,5,...,0.994872,-0.885480,-0.974353,0.697806,1.404173,0.571638,0.764710,-0.028688,2.19,2.148941
46,Kevin De Bruyne,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,0.500000,0.647059,0,33,11,...,-1.585720,1.134928,1.102921,0.488569,-1.951124,0.012660,-1.263435,-1.009407,1.96,2.127938
126,Mateo Kovačić,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,0.045455,0.058824,0,22,6,...,0.865842,1.137837,1.386941,-1.839196,2.000739,-0.126937,0.659323,-0.969915,2.32,2.103011


In [12]:
midfielder_min = np.min(midfielder_df['예측PPG'].tolist())
midfielder_min

1.014601469039917

In [13]:
midfielder_df['WAR'] = midfielder_df['예측PPG'].map(lambda x : x-midfielder_min+1)

In [14]:
midfielder_df.sort_values(by='WAR',ascending=False).head(60)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG,WAR
123,Saúl,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,0.409091,0.411765,21/7/5 - 22/6/30,31,6,...,0.960379,0.814369,0.200867,-5.066062e-02,0.062356,0.495165,-1.088392,2.11,2.291662,2.277060
124,N'Golo Kanté,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,0.272727,0.676471,0,34,10,...,0.972016,0.823434,-0.165298,-2.778275e-01,-0.496519,-1.347629,-0.858021,2.00,2.158232,2.143631
121,Hakim Ziyech,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,0.227273,0.176471,0,21,5,...,-0.885480,-0.974353,0.697806,1.404173e+00,0.571638,0.764710,-0.028688,2.19,2.148941,2.134340
46,Kevin De Bruyne,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,0.500000,0.647059,0,33,11,...,1.134928,1.102921,0.488569,-1.951124e+00,0.012660,-1.263435,-1.009407,1.96,2.127938,2.113336
126,Mateo Kovačić,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,0.045455,0.058824,0,22,6,...,1.137837,1.386941,-1.839196,2.000739e+00,-0.126937,0.659323,-0.969915,2.32,2.103011,2.088409
88,Bukayo Saka,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,0.590909,0.264706,0,26,4,...,-1.353854,-1.358082,0.697806,7.990828e-01,1.556755,0.247567,0.527492,1.91,2.047103,2.032502
128,Jorginho,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,0.090909,0.235294,0,32,10,...,1.110201,0.824944,0.462414,1.470545e+00,-0.908458,-0.097211,-0.927132,2.08,2.037688,2.023086
44,Jack Grealish,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,0.227273,0.088235,0,10,3,...,0.588007,0.521285,0.070094,1.409470e+00,0.308666,-1.446104,-0.463101,2.06,2.012128,1.997527
99,Tom Davies,2021-2022,미드필더,에버튼,잉글랜드 프리미어 리그,0.181818,0.088235,0,11,9,...,1.350206,1.415646,-0.479154,1.951963e+00,-0.401788,-0.015532,-0.923841,1.56,2.001708,1.987107
34,Thiago,2021-2022,미드필더,리버풀,잉글랜드 프리미어 리그,0.000000,0.117647,0,13,5,...,1.936401,2.116631,0.200867,1.239650e+00,-1.470545,-0.486994,-1.048899,2.10,1.966857,1.952255


In [15]:
midfielder_df.loc[:,outlier_features] = midfielder_qt.inverse_transform(midfielder_df.loc[:,outlier_features])
midfielder_df.loc[:,nonoutlier_features] = midfielder_ss.inverse_transform(midfielder_df.loc[:,nonoutlier_features])
midfielder_df.loc[:,order_features] = midfielder_mm.inverse_transform(midfielder_df.loc[:,order_features])

In [16]:
midfielder_df[['이름','시즌','도움', '경기당 유효 슈팅/90', '경기당 패스 시도/90', '반칙 수/90', '태클 성공/90','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,시즌,도움,경기당 유효 슈팅/90,경기당 패스 시도/90,반칙 수/90,태클 성공/90,WAR
123,Saúl,2021-2022,14.0,1.00,51.83,0.79,2.12,2.277060
124,N'Golo Kanté,2021-2022,23.0,0.29,51.91,1.08,1.95,2.143631
121,Hakim Ziyech,2021-2022,6.0,1.30,39.14,0.78,3.37,2.134340
46,Kevin De Bruyne,2021-2022,22.0,0.80,53.03,0.88,1.08,2.113336
126,Mateo Kovačić,2021-2022,2.0,0.84,53.05,1.03,4.61,2.088409
88,Bukayo Saka,2021-2022,9.0,1.23,35.92,1.05,2.81,2.032502
128,Jorginho,2021-2022,8.0,0.45,52.86,1.01,3.49,2.023086
44,Jack Grealish,2021-2022,3.0,0.83,49.27,1.01,3.38,1.997527
99,Tom Davies,2021-2022,3.0,0.62,54.51,1.44,4.45,1.987107
34,Thiago,2021-2022,4.0,0.32,58.54,0.87,3.19,1.952255
